<a href="https://colab.research.google.com/github/AndreuSerraSastre/03MAIR---Algoritmos-de-Optimizacion-2023/blob/main/Algoritmos_AG_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Andreu Serra Sastre
##GIT https://github.com/AndreuSerraSastre/03MAIR---Algoritmos-de-Optimizacion-2023.git

#Carga de librerias

In [1]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.9 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


#Carga de los datos del problema

In [2]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random                     #Para generar valores aleatorios

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos



In [3]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())



In [4]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html

#dir(problem)

15

#Funcionas basicas


In [7]:

#Funcionas basicas
###############################################################################

#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)


#BUSQUEDA ALEATORIA

In [22]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  #N es el numero de iteraciones
  Nodos = list(problem.get_nodes())

  mejor_solucion = []
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto

  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)

    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia


  print("Mejor solución:" , mejor_solucion)
  print("Distancia     :" , mejor_distancia)
  return mejor_solucion


#Busqueda aleatoria con 100000 iteraciones
solucion = busqueda_aleatoria(problem, 100000)

Mejor solución: [0, 33, 38, 23, 12, 5, 29, 39, 30, 32, 7, 1, 34, 22, 20, 35, 31, 13, 19, 16, 37, 17, 36, 14, 15, 25, 3, 27, 8, 11, 21, 10, 28, 9, 41, 18, 26, 2, 6, 4, 24, 40]
Distancia     : 3223


#BUSQUEDA LOCAL

In [23]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  mejor_solucion = []
  mejor_distancia = float('inf')
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

print("Distancia Solucion Incial:" , distancia_total(solucion, problem))

nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))


Distancia Solucion Incial: 3223
Distancia Mejor Solucion Local: 2992


In [33]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []

  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)
  print("Distancia inicial     :" , mejor_distancia)

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)

    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)
      return mejor_solucion

    solucion_referencia = vecina


sol = busqueda_local(problem)

Distancia inicial     : 4686
En la iteracion  39 , la mejor solución encontrada es: [0, 1, 6, 26, 18, 10, 8, 41, 23, 9, 21, 40, 24, 39, 22, 38, 29, 30, 33, 34, 28, 27, 2, 5, 13, 19, 14, 16, 15, 37, 7, 17, 31, 36, 35, 20, 32, 3, 4, 12, 11, 25]
Distancia     : 1641


In [64]:
###############################################################################
# MEJORA BUSQUEDA LOCAL
###############################################################################

def genera_vecina(solucion, problem):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos)
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina < distancia_total(solucion, problem):
        return vecina  # Devolvemos la primera vecina que mejore la solución actual
  return solucion  # Si ninguna vecina mejora la solución actual, la devolvemos sin cambios


def busqueda_local(problem, num_vecinos=5, max_iteraciones_sin_mejora=10):
    mejor_solucion = []

    # Generar una solucion inicial de referencia (aleatoria)
    solucion_referencia = crear_solucion(Nodos)
    mejor_distancia = distancia_total(solucion_referencia, problem)

    iteraciones_sin_mejora = 0
    while True:
        for _ in range(num_vecinos):
            vecina = genera_vecina(solucion_referencia, problem)

            distancia_vecina = distancia_total(vecina, problem)

            # Si la vecina mejora, actualizamos la referencia y restablecemos el contador
            if distancia_vecina < mejor_distancia:
                mejor_solucion = vecina
                mejor_distancia = distancia_vecina
                solucion_referencia = vecina
                iteraciones_sin_mejora = 0
            else:
                iteraciones_sin_mejora += 1

            # Si hemos superado el número máximo de iteraciones sin mejora, terminamos la búsqueda
            if iteraciones_sin_mejora >= max_iteraciones_sin_mejora:
                return mejor_solucion

def busqueda_local_multiple_creacion(problem, num_intentos=10, num_vecinos=5, max_iteraciones_sin_mejora=10):
    mejor_distancia = float('inf')
    mejor_sol = []

    for i in range(num_intentos):
        print('#', i)
        sol = busqueda_local(problem, num_vecinos, max_iteraciones_sin_mejora)
        distancia = distancia_total(sol, problem)

        if distancia < mejor_distancia:
            mejor_distancia = distancia
            mejor_sol = sol
            print('Mejor distancia ' , distancia)

    return mejor_sol

sol = busqueda_local_multiple_creacion(problem)
print("La mejor solución encontrada es:" , sol)
print("Distancia     :" , distancia_total(sol, problem))

# 0
Mejor distancia  1795
# 1
Mejor distancia  1722
# 2
Mejor distancia  1710
# 3
# 4
# 5
Mejor distancia  1587
# 6
# 7
# 8
# 9
La mejor solución encontrada es: [0, 27, 28, 30, 29, 2, 3, 4, 6, 5, 13, 26, 18, 8, 21, 24, 40, 23, 41, 25, 11, 12, 10, 9, 39, 22, 38, 34, 33, 20, 32, 19, 14, 16, 15, 37, 17, 36, 35, 31, 7, 1]
Distancia     : 1587
